In [2]:
!pip install -Uqq lxml

In [25]:
from bs4 import BeautifulSoup 
import urllib.request
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context
html_link = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datacurrent.html"

with urllib.request.urlopen(html_link) as url:
    s = url.read()
    # I'm guessing this would output the html source code ?
    soup = BeautifulSoup(s,"lxml")

In [34]:
html_table = soup.find_all("table")

In [50]:
ht = html_table[1]

In [56]:
a = ht.find_all('a')

In [81]:
req_href = {"US":[],"Europe":[],"Japan":[],"AUS_NZ_CANADA":[],"Emerging":[],"China":[],"India":[],"Global":[]}

for i in a:
    name = i.get_text().strip()
    try:
        href_attr = i['href']
        if href_attr.endswith('.xls'):
            if "US" in name:
                req_href["US"].append(href_attr)
            elif "Europe" in name:
                req_href["Europe"].append(href_attr)
            elif "Japan" in name:
                req_href["Japan"].append(href_attr)
            elif "Aus" in name:
                req_href['AUS_NZ_CANADA'].append(href_attr)
            elif "Emerging" in name:
                req_href['Emerging'].append(href_attr)
            elif "China" in name:
                req_href['China'].append(href_attr)
            elif "India" in name:
                req_href['India'].append(href_attr)
            elif "Global" in name: 
                req_href['Global'].append(href_attr)
    except:
        pass

In [85]:
import requests
ssl._create_default_https_context = ssl._create_stdlib_context

dls = req_href['US'][0]
resp = requests.get(dls,verify=False)

output = open('test.xls', 'wb')
output.write(resp.content)
output.close()

/home/athekunal/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pages.stern.nyu.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [88]:
import os
os.makedirs("DATA",exist_ok=True)
for country,excel_files in req_href.items():
    country_path = os.path.join("DATA",country) 
    os.makedirs(country_path,exist_ok=True)
    for file in excel_files:
        file_name = file.split("/")[-1].split(".")[0]
        full_file_name = os.path.join(country_path,f"{file_name}.xls")
        resp = requests.get(dls,verify=False)
        output = open(full_file_name, 'wb')
        output.write(resp.content)
        output.close()

In [94]:
for country in os.listdir("DATA"):
    dir_len = len(os.listdir(os.path.join("DATA",country)))
    country_len = len(req_href[country])
    print(f'DIR LEN = {dir_len} and ACTUAL LEN = {country_len}')

DIR LEN = 29 and ACTUAL LEN = 29
DIR LEN = 29 and ACTUAL LEN = 29
DIR LEN = 29 and ACTUAL LEN = 29
DIR LEN = 29 and ACTUAL LEN = 29
DIR LEN = 29 and ACTUAL LEN = 29
DIR LEN = 29 and ACTUAL LEN = 29
DIR LEN = 24 and ACTUAL LEN = 24
DIR LEN = 29 and ACTUAL LEN = 29


In [118]:
# !pip install -Uqq xlrd
!pip install -Uqq openpyxl

In [2]:
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_rows', 50)

dir = "DATA"
for country in os.listdir(dir):
    print(country)
    file_name = os.path.join(dir,country)
    for excel_file in tqdm(os.listdir(file_name)):
        full_file_name = os.path.join(file_name,excel_file)
        xls = pd.read_excel(full_file_name,sheet_name="Industry Averages")
        # info = pd.read_excel(full_file_name,sheet_name="Variables & FAQ")
        xls.dropna(inplace=True)
        new_header = xls.iloc[0] #grab the first row for the header
        xls = xls[1:] #take the data less the header row
        xls.reset_index(inplace=True,drop=True)
        xls.columns = new_header #set the header row as the df header
        xls.to_excel(full_file_name+"x",sheet_name="Industry Averages",index=False)
        # info.to_excel(full_file_name+"x",sheet_name="Variables & FAQ",index=False)